In [1]:
import geopandas as gpd
import numpy as np
import os
import matplotlib.pyplot as plt
from shapely.geometry import shape, LineString, mapping
from shapely.ops import unary_union, polygonize
import glob
import pandas as pd
import csv
from datetime import datetime
import time

c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
Q:\UserTemp\ubach\AppData\Local\Temp\3\ipykernel_39336\3081711937.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https:

In [3]:
# Define engines
engines = {
    'fiona': {'engine': 'fiona'},
    'pyogrio': {'engine': 'pyogrio'},
    'pyogrio+arrow': {'engine': 'pyogrio', 'use_arrow': True}
          
}

In [4]:
# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
step3_f = os.path.join(indata_f, 'OutputData', 'step3_SA', 'Dissolved')
outdata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\OutputData'
ua_data_f = r'A:\Copernicus\UrbanAtlas\UrbanAtlas\UA2018'
# Load urban center delineation
path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2B3B4Copy.shp') 
urban_centers = gpd.read_file(path)

# NOISE DATA
# Load agglomerations delineations
agls_file_path = os.path.join(indata_f, 'NoiseData', 'DF1_5_Agglomerations_20240429.gpkg')
# Read the GeoPackage file
agls = gpd.read_file(agls_file_path, layer = 'dbo.DF15_AgglomerationSource_Valid_LatestDelivery', 
                     **engines['pyogrio+arrow'],columns=['agglomerationId_identifier', 'agglomerationName_nameEng', 'geometry'])

# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

# GQA
path2FinalGQA = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\OutputData\step2_GQA_Final'

# FINAL OUTPUT TABLE
csv_name = 'GQATotals_DGREGIO_v01.csv'
csv_path = os.path.join(outdata_f, 'OutTables', csv_name)

In [5]:
csv_name = 'GQATotals_DGREGIO_v02.csv'
csv_path = os.path.join(outdata_f, 'OutTables', csv_name)
SA_ls = ['GEOSTAT21_222',
 'GEOSTAT21_297',
 'GEOSTAT21_334',
 'GEOSTAT21_363',
 'GEOSTAT21_405',
 'GEOSTAT21_408',
 'GEOSTAT21_511',
 'GEOSTAT21_650', 
 'GEOSTAT21_901',
 'GEOSTAT21_925']

# 222

In [ ]:
uc_code= 'GEOSTAT21_222'
GQA_path = os.path.join(path2FinalGQA, f'{uc_code}_finalGQA.shp')
GQA = gpd.read_file(GQA_path)
GQA = GQA.reset_index().rename(columns={'index': 'GQA_FID'})

In [22]:
GQA.columns.to_list()

['GQA_FID',
 'GQA_FID',
 'country',
 'fua_name',
 'fua_code',
 'HDENS_2011',
 'code_2018',
 'class_2018',
 'noisy',
 'area_m2',
 'area_ha',
 'area_km2',
 'GQA_m2',
 'geometry']

In [23]:
GQA2 = GQA

In [24]:
GQA2.drop(columns=['GQA_FID'], inplace=True)

In [26]:
GQA2  = GQA2.reset_index().rename(columns={'index': 'GQA_FID'})
GQA2.columns.to_list()

['GQA_FID',
 'country',
 'fua_name',
 'fua_code',
 'HDENS_2011',
 'code_2018',
 'class_2018',
 'noisy',
 'area_m2',
 'area_ha',
 'area_km2',
 'GQA_m2',
 'geometry']

In [21]:
GQA = GQA[['GQA_FID',
 'country',
 'fua_name',
 'fua_code',
 'HDENS_2011',
 'code_2018',
 'class_2018',
 'noisy',
 'area_m2',
 'area_ha',
 'area_km2',
 'GQA_m2',
 'geometry']]

In [27]:
GQA2.to_file(GQA_path, driver='ESRI Shapefile')

In [7]:
SA_f = os.path.join(indata_f, 'OutputData', 'step3_SA')
file_path_SA = os.path.join(SA_f, f'{uc_code}_SA.shp')
SA = gpd.read_file(file_path_SA)

In [8]:
SA.head()

,FacilityID,Name,FromBreak,ToBreak,Shape_Leng,Shape_Area,FID_1,geometry
0,118,Location 118 : 0 - 400,0.0,400.0,2793.127011,44175.0,13,"POLYGON ((3940032.287 3225118.752, 3940037.287..."
1,126,Location 126 : 0 - 400,0.0,400.0,2793.127011,44175.0,13,"POLYGON ((3940032.287 3225118.752, 3940037.287..."
2,115,Location 115 : 0 - 400,0.0,400.0,3227.610373,38887.5,12,"POLYGON ((3940052.287 3225148.752, 3940057.287..."
3,124,Location 124 : 0 - 400,0.0,400.0,3238.084044,39025.0,13,"POLYGON ((3940062.287 3225143.752, 3940062.287..."
4,125,Location 125 : 0 - 400,0.0,400.0,2857.684754,44137.5,13,"POLYGON ((3940027.287 3225128.752, 3940042.287..."


In [11]:
SA.FID_1.max()

29

In [9]:
dissolved_gdf = SA.dissolve(by="FID_1").reset_index()

In [10]:
dissolved_gdf

,FID_1,geometry,FacilityID,Name,FromBreak,ToBreak,Shape_Leng,Shape_Area
0,0,"POLYGON ((3939402.287 3226368.752, 3939405.923...",7,Location 7 : 0 - 400,0.0,400.0,4291.586109,137725.0
1,1,"POLYGON ((3939442.287 3225603.752, 3939421.870...",10,Location 10 : 0 - 400,0.0,400.0,6503.197989,218637.5
2,2,"POLYGON ((3939505.620 3226175.418, 3939522.287...",32,Location 32 : 0 - 400,0.0,400.0,5556.720822,190187.5
3,3,"POLYGON ((3939266.892 3225516.383, 3939267.287...",39,Location 39 : 0 - 400,0.0,400.0,4675.505455,87625.0
4,4,"POLYGON ((3940221.404 3225969.340, 3940281.012...",53,Location 53 : 0 - 400,0.0,400.0,4412.929354,145137.5
5,5,"POLYGON ((3940057.287 3226218.752, 3940047.287...",57,Location 57 : 0 - 400,0.0,400.0,4243.953243,145137.5
6,6,"POLYGON ((3939647.287 3225523.752, 3939652.287...",61,Location 61 : 0 - 400,0.0,400.0,3299.987825,70675.0
7,7,"POLYGON ((3939502.287 3226198.752, 3939502.287...",83,Location 83 : 0 - 400,0.0,400.0,4715.761814,139925.0
8,8,"POLYGON ((3939376.215 3226022.770, 3939392.287...",86,Location 86 : 0 - 400,0.0,400.0,3746.557326,99162.5
9,9,"POLYGON ((3940346.572 3226108.038, 3940352.287...",95,Location 95 : 0 - 400,0.0,400.0,5187.709373,195125.0


In [14]:
SAd_f = os.path.join(indata_f, 'OutputData', 'step3_SA', 'Dissolved')
file_path_SAd = os.path.join(SAd_f, f'{uc_code}_SAd.shp')
dissolved_gdf.to_file(file_path_SAd, driver='ESRI Shapefile')
SA_GQA_total = gpd.read_file(file_path_SAd)

### 901

In [28]:
uc_code= 'GEOSTAT21_901'
GQA_path = os.path.join(path2FinalGQA, f'{uc_code}_finalGQA.shp')
GQA = gpd.read_file(GQA_path)
GQA = GQA.reset_index().rename(columns={'index': 'GQA_FID'})

In [29]:
GQA

,GQA_FID,country,fua_name,fua_code,HDENS_2011,code_2018,class_2018,noisy,area_m2,area_ha,area_km2,GQA_FID,geometry
0,0,FI,Helsinki,FI001L3,None,31000,Forests,0.0,16457.559538,1.65,0.02,0,"POLYGON ((5143167.880 4203011.980, 5143167.985..."
1,1,FI,Helsinki,FI001L3,None,31000,Forests,0.0,12930.504617,1.29,0.01,1,"POLYGON ((5142732.129 4203023.258, 5142817.339..."
2,2,FI,Helsinki,FI001L3,None,31000,Forests,0.0,36151.269317,3.62,0.04,2,"POLYGON ((5142917.415 4203091.233, 5142917.518..."
3,3,FI,Helsinki,FI001L3,None,31000,Forests,0.0,143860.442376,14.39,0.14,3,"POLYGON ((5141025.939 4203574.851, 5141081.213..."
4,4,FI,Helsinki,FI001L3,None,31000,Forests,0.0,9459.284556,0.95,0.01,4,"POLYGON ((5141589.131 4203628.695, 5141580.931..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1869,1869,FI,Helsinki,FI001L3,None,31000,Forests,0.0,2120.584284,0.21,0.00,3689,"POLYGON ((5133013.189 4209399.542, 5133012.973..."
1870,1870,FI,Helsinki,FI001L3,None,31000,Forests,0.0,53016.470113,5.30,0.05,3690,"POLYGON ((5132733.274 4209457.066, 5132733.357..."
1871,1871,FI,Helsinki,FI001L3,None,31000,Forests,0.0,7793.625035,0.78,0.01,3691,"POLYGON ((5132492.335 4209430.241, 5132495.893..."
1872,1872,FI,Helsinki,FI001L3,None,31000,Forests,0.0,29956.410373,3.00,0.03,3692,"POLYGON ((5131880.156 4209365.840, 5131833.705..."


### 650

In [30]:
uc_code= 'GEOSTAT21_650'
GQA_path = os.path.join(path2FinalGQA, f'{uc_code}_finalGQA.shp')
GQA = gpd.read_file(GQA_path)
GQA = GQA.reset_index().rename(columns={'index': 'GQA_FID'})

In [31]:
GQA

,GQA_FID,country,fua_name,fua_code,HDENS_2011,code_2018,class_2018,noisy,area_m2,area_ha,area_km2,GQA_FID,geometry
0,0,ES,Bilbao,ES019L3,GEOSTAT11_650,14100,Green urban areas,0.0,397.933722,0.04,0.00,0,"POLYGON ((3274179.906 2327662.600, 3274179.916..."
1,1,ES,Bilbao,ES019L3,GEOSTAT11_650,14100,Green urban areas,0.0,27277.819355,2.73,0.03,1,"POLYGON ((3274581.477 2327419.349, 3274582.289..."
2,2,ES,Bilbao,ES019L3,GEOSTAT11_650,14100,Green urban areas,0.0,63835.952296,6.38,0.06,2,"POLYGON ((3275268.037 2328007.008, 3275274.899..."
3,3,ES,Bilbao,ES019L3,GEOSTAT11_650,14100,Green urban areas,0.0,127.195249,0.01,0.00,3,"POLYGON ((3274873.139 2327837.237, 3274878.555..."
4,4,ES,Bilbao,ES019L3,GEOSTAT11_650,14100,Green urban areas,0.0,1006.251388,0.10,0.00,4,"POLYGON ((3274712.414 2327912.835, 3274699.774..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,108,ES,Bilbao,ES019L3,GEOSTAT11_650,14100,Green urban areas,0.0,18753.665759,1.88,0.02,168,"POLYGON ((3270547.053 2334132.676, 3270541.909..."
109,109,ES,Bilbao,ES019L3,GEOSTAT11_650,14100,Green urban areas,0.0,6063.007485,0.61,0.01,169,"POLYGON ((3270812.577 2334279.447, 3270817.532..."
110,110,ES,Bilbao,ES019L3,GEOSTAT11_650,14100,Green urban areas,0.0,21132.436851,2.11,0.02,170,"POLYGON ((3270882.162 2334409.520, 3270913.080..."
111,111,ES,Bilbao,ES019L3,GEOSTAT11_650,14100,Green urban areas,0.0,9590.692434,0.96,0.01,171,"POLYGON ((3270648.302 2334746.198, 3270638.096..."


### 334

In [35]:
uc_code = 'GEOSTAT21_334'
group_combinations = codes.query(f'HDENS_CLST=="{uc_code}"')[['HDENS_CLST', 'UA2018']].drop_duplicates()
group_combinations = group_combinations.groupby(['HDENS_CLST', 'UA2018']).size().reset_index(name='count')
ua_ls = group_combinations['UA2018'].tolist()

In [37]:
codes.query(f'HDENS_CLST=="{uc_code}"')[['HDENS_CLST', 'UA2018']].drop_duplicates()

,HDENS_CLST,UA2018
141,GEOSTAT21_334,DE516L0_SOLINGEN_UA2018_v013
142,GEOSTAT21_334,DE011L1_DUSSELDORF_UA2018_v013


In [36]:
ua_ls

['DE011L1_DUSSELDORF_UA2018_v013', 'DE516L0_SOLINGEN_UA2018_v013']